In [ ]:
import pandas as pd

from neuralforecast.models import *
from neuralforecast.core import NeuralForecast
from neuralforecast.losses.pytorch import MAE

In [ ]:
HORIZON = 12
LOSS = MAE()

In [ ]:
Y_df = pd.read_parquet('s3://timenet/monash_1.parquet')

In [ ]:
Y_df.dtypes

In [ ]:
Y_df

In [ ]:
Y_df.dataset.value_counts()

In [ ]:
Y_df.frequency.value_counts()

In [ ]:
model = NHITS(h=HORIZON,
                input_size=HORIZON,
                stack_types=3*['identity'],
                mlp_units=3 * [[1024, 1024, 1024, 1024]],
                n_blocks=3*[10],
                n_pool_kernel_size=3*[1],
                n_freq_downsample=[6, 2, 1],
                loss=LOSS,
                learning_rate=1e-4,
                early_stop_patience_steps=5,
                val_check_steps=500,
                scaler_type='minmax1',
                max_steps=1000, # Partition steps
                batch_size=256,
                windows_batch_size=1024,
                random_seed=1)

In [ ]:
nf = NeuralForecast(models=[model], freq='10min')

In [ ]:
for partition in range(16):
    print(f'Partition {partition}')
    Y_df = pd.read_parquet('s3://timenet/monash_1.parquet')
    Y_df = Y_df[['unique_id', 'ds', 'y']]
    nf.fit(df=Y_df, use_init_models=False, val_size=HORIZON)